In [1]:
import pandas as pd
import hashlib
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import textwrap
import glob
import json
def create_hash(str):
    return hashlib.sha256(str.encode('utf-8')).hexdigest().upper()

dash_data = dict()


evaluations_dir = 'evaluation_results'
for f in glob.glob('{}/*.csv'.format(evaluations_dir)):
    df = pd.read_csv(f)
    for index, row in df.iterrows():
        ifp_h = create_hash(row['ifp'])
        if ifp_h not in dash_data:
            dash_data[ifp_h] = dict()
        d = dash_data[ifp_h]
        sentence = row['sentence_text']
        s_h = create_hash(sentence)
        if s_h not in d:
            d[s_h] = dict()
            d[s_h]['score'] = row['score']
            d[s_h]['relevance'] = list()
            d[s_h]['sentence'] = sentence
            d[s_h]['ifp'] = row['ifp']
        d[s_h]['relevance'].append(row['relevance'])
        
for i in list(dash_data):
    data = dash_data[i]
    for s in list(data):
        data[s]['relevance'] = sum(data[s]['relevance']) / float(len(data[s]['relevance']))

plot_dash_data = dict()
for key in list(dash_data):
    if key not in plot_dash_data:
        plot_dash_data[key] = list()
    for s in list(dash_data[key]):
        plot_dash_data[key].append(dash_data[key][s])


In [ ]:
app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(
        id='score-vs-relevance',
        figure={
            'data': [
                go.Scatter(
                    x=[g['score'] for g in plot_dash_data[key]],
                    y=[g['relevance'] for g in plot_dash_data[key]],
                    text=[g['sentence'] for g in plot_dash_data[key]],
                    mode='markers',
                    opacity=0.7,
                    marker={
                        'size': 15,
                        'line': {'width': 0.5, 'color': 'white'}
                    },
                    name='<br>'.join(textwrap.wrap(plot_dash_data[key][0]['ifp'], width=66)),
                ) for key in plot_dash_data.keys()
            ],
            'layout': go.Layout(
                xaxis={'type': 'log', 'title': 'Score'},
                yaxis={'title': 'Relevance'},
                margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                legend={'xanchor':'right', 'yanchor': 'top', 'font': {'size': 8}},
                hovermode='closest'
            )
        }
    )
])
app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Sep/2018 17:09:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2018 17:09:33] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2018 17:09:33] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Sep/2018 17:09:33] "GET /favicon.ico HTTP/1.1" 200 -
